In [ ]:
!pip install torch==1.4.0

ERROR: Could not find a version that satisfies the requirement torch==1.4.0 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0)
ERROR: No matching distribution found for torch==1.4.0


In [ ]:
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Parameter

# Check whether GPU is available and can be used
# if CUDA is found then device is set accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if not torch.cuda.is_available():
    print("Consider changing your run-time to GPU or training will be slow.")

## The data: Shakespeare's sonnets

Shakespeare's sonnets can be found at the following URL featuring all of his works: http://shakespeare.mit.edu/

For convenience reasons we have extracted all the plain text of the sonnets: https://ocw.mit.edu/ans7870/6/6.006/s08/lecturenotes/files/t8.shakespeare.txt into a separate textfile and have added it to the class' repository. We will thus download it from there:

In [ ]:
!wget https://raw.githubusercontent.com/ccc-frankfurt/Practical_ML_SS21/master/week06/sonnets.txt

--2023-10-13 06:44:13--  https://raw.githubusercontent.com/ccc-frankfurt/Practical_ML_SS21/master/week06/sonnets.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 94081 (92K) [text/plain]
Saving to: ‘sonnets.txt’

sonnets.txt         100%[===================>]  91.88K  --.-KB/s    in 0.01s   

2023-10-13 06:44:13 (7.31 MB/s) - ‘sonnets.txt’ saved [94081/94081]



We can open the text file and print an excerpt.

In [ ]:
# Open shakespeare text file and read the data
with open('sonnets.txt', 'r') as f:
    text = f.read()

# print an excerpt of the text
print(text[:200])

From fairest creatures we desire increase,
That thereby beauty's rose might never die,
But as the riper should by time decease,
His tender heir might bear his memory:
But thou contracted to thine own 


As we are interested in a character based neural network, we will now create a mapping from the characters to numbers so that we can do our matrix calculations with numerical data. One such way is to simply replace every character with the corresponding integer in an alphabetical sequence. If we print our excerpt, we can now see the corresponding numerical values of each character.

In [ ]:
# We create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to integers
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# Encode the text
encoded = np.array([char2int[ch] for ch in text])

# Again showing the excerpt, but this time as integers
encoded[:200]

array([ 8, 14, 52, 48, 46, 42, 16,  4, 14, 12, 30, 23, 46, 53, 14, 12, 16,
       23,  9, 14, 12, 30, 46, 39, 12, 46, 13, 12, 30,  4, 14, 12, 46,  4,
        2, 53, 14, 12, 16, 30, 12, 57,  6,  3, 59, 16, 23, 46, 23, 59, 12,
       14, 12,  7, 25, 46,  7, 12, 16,  9, 23, 25, 22, 30, 46, 14, 52, 30,
       12, 46, 48,  4, 47, 59, 23, 46,  2, 12, 10, 12, 14, 46, 13,  4, 12,
       57,  6, 55,  9, 23, 46, 16, 30, 46, 23, 59, 12, 46, 14,  4, 18, 12,
       14, 46, 30, 59, 52,  9, 15, 13, 46,  7, 25, 46, 23,  4, 48, 12, 46,
       13, 12, 53, 12, 16, 30, 12, 57,  6, 21,  4, 30, 46, 23, 12,  2, 13,
       12, 14, 46, 59, 12,  4, 14, 46, 48,  4, 47, 59, 23, 46,  7, 12, 16,
       14, 46, 59,  4, 30, 46, 48, 12, 48, 52, 14, 25, 24,  6, 55,  9, 23,
       46, 23, 59, 52,  9, 46, 53, 52,  2, 23, 14, 16, 53, 23, 12, 13, 46,
       23, 52, 46, 23, 59,  4,  2, 12, 46, 52, 39,  2, 46])

### Data loader: batching

We now have our entire text file encoded as integers, which serves as our dataset. Next, we will need to define our data loader, mainly the part that is missing, the random sampling of batches. Let us define this method:

In [ ]:
# Defining method to make mini-batches for training
def get_batches(arr, batch_size, seq_length):
    # determine the flattened batch size, i.e. sequence length times batch size
    batch_size_total = batch_size * seq_length
    # total number of batches we can make
    n_batches = len(arr)//batch_size_total

    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))

    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

### Targets/Labels

We will be treating our problem as a classification task, where given an input the task is to predict the likelihood of the next character, i.e. we choose the class/character with the highest probability of a SoftMax output. Our model's output is thus a vector containing a probability for each unique character.

Since we want to be able to feed our model's output back as input for the next time step, we should also give the network a one-hot encoded character as the input instead of just an integer, similar to what we have seen on our lecture's last slide.
This way the network gets as input a one-hot vector of length corresponding to the number of total unique characters and predicts the likelihood for each character as output (for the next character in the sequence). We will thus write a function that converts our encoded characters from integers to one-hot vectors.

In [ ]:
def one_hot_encode(arr, n_labels):

    # Initialize the the encoded array
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)

    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.

    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))

    return one_hot

## A simple RNN

We will start with writing a simple RNN in PyTorch. To get a better understanding of how the RNN model works, we will not be using PyTorch's convenience RNN implementation, but write the main portion by hand ourselves. We will later use the convenience functions for the much more complicated LSTMs.

Note that we could in principle do the same thing in pure Numpy but the advantage of implementing the forward logic in PyTorch is that we can use the automatic differentation for our backward pass and we do not need to implement the backpropagation through time ourselves.

What we will learn here is:
1. How to write a recurrent neural network (the forward pass)
2. How to implement custom mathematical equations in the forward pass of a PyTorch model and leverage the automatic backward.

In [ ]:
class RNN(nn.Module):
    def __init__(self, chars, device, hidden_sz, drop_prob=0.5):
        super().__init__()

        self.device = device

        # creating character dictionaries
        # we already have this code on the top, but giving it to our model
        # will be convenient for doing predictions later
        # i.e. doing conversions from text to integers to one-hot & vice-versa
        self.n_chars = len(chars)
        self.int2char = dict(enumerate(chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}

        self.hidden_sz = hidden_sz

        # Note that this class inherits from the torch neural network class
        # Instead of using a pre-built function we will write the math ourselves
        # For this reason we will first need to define "Parameters()", that
        # the PyTorch graph keeps track of and can optimize. In other words,
        # let's give our class the weights & the bias that the RNN will need.
        self.weight_ih = Parameter(torch.Tensor(self.n_chars, self.hidden_sz))
        self.weight_hh = Parameter(torch.Tensor(self.hidden_sz, self.hidden_sz))
        self.bias_hh = Parameter(torch.Tensor(self.hidden_sz))

        # Now that we have defined the RNN cell, let us define the output layer
        # We will use a dropout layer to prevent overfitting and then
        # follow with a conventional linear layer (matrix multiplication) that
        # maps the RNN cell's output (the hidden state of the network) to the
        # class output. Remembert that the class output corresponds to a
        # vector of length of unique characters.

        # define a dropout layer
        self.dropout = nn.Dropout(drop_prob)

        # define the final, fully-connected output layer. We can use a
        # PyTorch nn function here (or you could add the corresponding math
        # below and assign an additional weight & bias at the top).
        # We can see that we can create very custom models this way
        self.fc = nn.Linear(self.hidden_sz, self.n_chars)

        # We have assigned the Parameters above, but we will need to also
        # initialize them. Let's write a function for that and initialize
        # our weights and bias.
        self.init_weights()

    def init_weights(self):
        nn.init.xavier_uniform_(self.weight_ih)
        nn.init.xavier_uniform_(self.weight_hh)
        nn.init.zeros_(self.bias_hh)

    def forward(self, x, h_t):
        """Assumes x is of shape (batch, sequence, feature)"""
        bs, seq_sz, _ = x.size()
        hidden_seq = []

        # Given an input and an initial hidden state, calculate the next hidden
        # state for each sequence element.
        # We append all the hidden states to a list (similar to a batch size)
        # so that we can concatenate them in the batch and feed them to our
        # last linear layer all in parallel to avoid looping through the final
        # output layer as there is no more dependence on other time steps.
        for t in range(seq_sz):
            x_t = x[:, t, :]
            h_t = torch.tanh(x_t @ self.weight_ih + h_t @ self.weight_hh + self.bias_hh)
            hidden_seq.append(h_t.unsqueeze(0))

        # Do the concatenation and reshaping for convenience
        hidden_seq = torch.cat(hidden_seq, dim=0)
        # reshape from shape (sequence, batch, feature) to (batch, sequence, feature)
        hidden_seq = hidden_seq.transpose(0, 1).contiguous()

        # Stack up the RNN outputs using view so that we can process the last
        # layer in parallel
        r_output = hidden_seq.contiguous().view(-1, self.hidden_sz)

        # pass through a dropout layer
        out = self.dropout(r_output)

        # Calculate fully connected layer output that yields our class vector
        out = self.fc(out)

        return out, h_t

    def init_hidden(self, batch_size=1):
        ''' Initializes hidden state '''
        # This is a convenience function so that we can initialize a hidden
        # state to zero when we start prediction on a sequence. Every further
        # step will then depend on the previous hidden state.

        # Create two new tensors with sizes batch_size x n_hidden,
        # initialized to zero for hidden the RNN's hidden state.
        weight = next(self.parameters()).data
        h_t = weight.new(batch_size, self.hidden_sz).zero_().to(device)

        return h_t


The only thing missing is our training loop. It will look very similar to everything we have previously written, with two main differences:

1. Our model is now also dependent on the hidden state and thus takes it as input and returns it as an additional output.
2. Because we are using a recurrent neural network we will need to give our "loss.backward()" a "retain_graph=True" flag in order for it to log the history and be able to compute the backpropagation through time

In [ ]:
# Declaring the train method
def train(model, data, device, optimizer, criterion, epochs=10, batch_size=10,
          seq_length=50, clip=5):
    model.train()

    for epoch in range(epochs):
        # initialize first hidden states with zeros
        h = model.init_hidden(batch_size)

        for x, y in get_batches(data, batch_size, seq_length):
            # One-hot encode our data, make them torch tensors & cast to device
            x = one_hot_encode(x, model.n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            inputs, targets = inputs.to(device), targets.to(device)

            # zero accumulated gradients
            model.zero_grad()

            # get the output and hidden state from the model
            output, h = model(inputs, h)

            # calculate the loss and perform backprop
            # because we have flattened our batch and sequence in the model to
            # be able to speed up the connection of the last fully-connected
            # layer we now also need to view/flatten our target here
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward(retain_graph=True)

            # we use an additional trick of clipping gradients to avoid
            # exploding gradients, which is a prominent problem in RNNs, just
            # as the opposite problem of vanishing gradients.
            nn.utils.clip_grad_norm_(model.parameters(), clip)
            optimizer.step()


        print("Epoch: {}/{}:".format(epoch + 1, epochs),
              "Loss: {:.4f}:".format(loss.item()))

In [ ]:
# Define the model
n_hidden=512
model = RNN(chars, device, n_hidden).to(device)

# Hyperparameters
batch_size = 128
seq_length = 100
epochs = 300 # start with 50 or similar if you are debugging
# train much longer if you want good results

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# train the model
train(model, encoded, device, optimizer, criterion, epochs=epochs,
      batch_size=batch_size, seq_length=seq_length)

Epoch: 1/300: Loss: 3.3650:
Epoch: 2/300: Loss: 4.8868:
Epoch: 3/300: Loss: 3.1997:
Epoch: 4/300: Loss: 3.1644:
Epoch: 5/300: Loss: 3.1470:
Epoch: 6/300: Loss: 3.1201:
Epoch: 7/300: Loss: 3.0993:
Epoch: 8/300: Loss: 3.0726:
Epoch: 9/300: Loss: 3.0464:
Epoch: 10/300: Loss: 3.0101:
Epoch: 11/300: Loss: 2.9749:
Epoch: 12/300: Loss: 2.9309:
Epoch: 13/300: Loss: 2.8849:
Epoch: 14/300: Loss: 2.8634:
Epoch: 15/300: Loss: 2.8088:
Epoch: 16/300: Loss: 2.7600:
Epoch: 17/300: Loss: 2.7497:
Epoch: 18/300: Loss: 2.7074:
Epoch: 19/300: Loss: 2.6691:
Epoch: 20/300: Loss: 2.6747:
Epoch: 21/300: Loss: 2.6148:
Epoch: 22/300: Loss: 2.5715:
Epoch: 23/300: Loss: 2.5435:
Epoch: 24/300: Loss: 2.5065:
Epoch: 25/300: Loss: 2.4917:
Epoch: 26/300: Loss: 2.4652:
Epoch: 27/300: Loss: 2.4451:
Epoch: 28/300: Loss: 2.4137:
Epoch: 29/300: Loss: 2.4136:
Epoch: 30/300: Loss: 2.3806:
Epoch: 31/300: Loss: 2.3623:
Epoch: 32/300: Loss: 2.3492:
Epoch: 33/300: Loss: 2.3662:
Epoch: 34/300: Loss: 2.3346:
Epoch: 35/300: Loss: 2.

You should observe the loss sinking consistently. In fact you can observe that the model training hasn't fully converged yet. If you feel like you want to spend the time later to see how
well you can get your RNN to perform, try training it for longer/until convergence.

Once we have trained the model it will be interesting to use it for prediction. To generate new content we would like to feed in an initial sequence or even just a single character and see what the model generates for the rest of the sequence conditioned on our input.

Let us write the logic for that:

In [ ]:
def predict(model, char, device, h=None, top_k=5):
        ''' Given a character & hidden state, predict the next character.
            Returns the predicted character and the hidden state.
        '''

        # tensor inputs
        x = np.array([[model.char2int[char]]])
        x = one_hot_encode(x, model.n_chars)
        inputs = torch.from_numpy(x).to(device)

        with torch.no_grad():
            # get the output of the model
            out, h = model(inputs, h)

            # get the character probabilities
            # move to cpu for further processing with numpy etc.
            p = F.softmax(out, dim=1).data.cpu()

            # get the top characters with highest likelihood
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()

            # select the likely next character with some element of randomness
            # for more variability
            p = p.numpy().squeeze()
            char = np.random.choice(top_ch, p=p/p.sum())

        # return the encoded value of the predicted char and the hidden state
        return model.int2char[char], h

In [ ]:
def sample(model, size, device, prime='A', top_k=None):
    # method to generate new text based on a "prime"/initial sequence.
    # Basically, the outer loop convenience function that calls the above
    # defined predict method.
    model.eval() # eval mode

    # Calculate model for the initial prime characters
    chars = [ch for ch in prime]
    with torch.no_grad():
        # initialize hidden with 0 in the beginning. Set our batch size to 1
        # as we wish to generate one sequence only.
        h = model.init_hidden(batch_size=1)
        for ch in prime:
            char, h = predict(model, ch, device, h=h, top_k=top_k)

        # append the characters to the sequence
        chars.append(char)

        # Now pass in the previous/last character and get a new one
        # Repeat this process for the desired length of the sequence to be
        # generated
        for ii in range(size):
            char, h = predict(model, chars[-1], device, h=h, top_k=top_k)
            chars.append(char)

    return ''.join(chars)

### Generating poems

We are now set to call our sample method with our trained model, a prime sequence and a desired sequence length to be generated.

In [ ]:
print(sample(model, 1000, device, prime='A', top_k=5))

A deast te th, and congant and wher wos ang yys's ercentes lo dans all beacine shef the paigeats an the har thou bayt rof tree,
Which art th t me tho  hall sot ond atr goen,
I shen whethants aid ain that llov's ain, my teme beathings foo the praised, afr alo buin thathe th sha e ous as ay nom soull ag ooll, st il prishat wiecur sie,
Bup sue too trol not is pusting bsake whel are thrisce wall noundse whes formy that rave morest mast thes list st me hing and pprnes, bud  ave beto bring,
That foow ho nor, in the ithingre woth thins and fyor serpenti plensere bule bous re fout tio geat,
And shal got un wate then fie for mise concestinghy braovy's gate llive wouthe terurwell ssall ghtieg and hors
And soandst and wilh grtser in the wredsings and ill whil th the beathy shave I steno, sor sone, eith doc il nor sur on wyolhin wellde swell yelref in thea sorten songees wind lave tore shate:
I hot of the roul wnon thas llo goulls of you whed thau nou here,
An henc muss fort such prois of illiss i

We can see that our RNN typically starts out correctly and sometimes is able to generate correct words but quickly goes on to generate junk as there is no long term dependencies.

We will now implement a PyTorch LSTM to see how to improve upon this.

## Long Short Term Memory (LSTM)

Let's take our recurrent neural network class that we have defined above and replace the simple RNN cell with one or even multiple stacked LSTM cells.

If you want to go for the challenge you can try implementing this by hand similarly to the RNN cell we have defined. However, if you don't want to go through the tour-de-force exercise, you can go on ahead and use PyTorch's "nn.LSTM()" convenience method: https://pytorch.org/docs/stable/nn.html#torch.nn.LSTM

You can try using a stack of 2 LSTM hidden layers to simply replace the RNN cell.

In [ ]:
class LSTM(nn.Module):

    def __init__(self, chars, device, n_hidden=256, n_layers=2, drop_prob=0.5):
        super().__init__()

        self.device = device

        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden

        # creating character dictionaries
        # we already have this code on the top, but giving it to our model
        # will be convenient for doing predictions later
        # i.e. doing conversions from text to integers to one-hot & vice-versa
        self.n_chars = len(chars)
        self.int2char = dict(enumerate(chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}

        # define the LSTM
        # we no longer need to care about wieght initialization as PyTorch
        # will handle this for us now.
        # When defining PyTorch's nn.LSTM() set "batch_first=True" to assign
        # the batch size to the first dimension (instead of the sequence) to
        # stay consistent with our RNN implementation and re-use our code.
        self.lstm = nn.LSTM(self.n_chars, n_hidden, n_layers,
                            dropout=drop_prob, batch_first=True)

        # define a dropout layer
        self.dropout = nn.Dropout(drop_prob)

        # define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, self.n_chars)

    def forward(self, x, hidden):
        ''' Forward pass through the network.
            The inputs are x, and the hidden & cell state in a tuple. '''

        # get the outputs and the new hidden states from the LSTM.
        # Note that the hidden variable now is a tuple of hidden and cell state
        # in contrast to the RNN that just had the hidden state.
        # Because we are using the PyTorch LSTM we do not need to implement
        # the loop anymore as the sequence will be handled internally.
        r_output, hidden = self.lstm(x, hidden)

        # pass through a dropout layer
        out = self.dropout(r_output)

        # Stack up the LSTM outputs using view so that we can process the last
        # layer in parallel
        out = out.contiguous().view(-1, self.n_hidden)

        # Calculate fully connected layer output that yields our class vector
        out = self.fc(out)

        return out, hidden

    def init_hidden(self, batch_size=1):
        ''' Initializes hidden state '''
        # This is a convenience function so that we can initialize the hidden
        # states to zero when we start prediction on a sequence. Every further
        # step will then depend on the previous hidden states (c and h).

        # Create a tuple of two new tensors with sizes
        # n_layers x batch_size x n_hidden, initialized to zero for the
        # LSTM hidden and cell states.
        weight = next(self.parameters()).data

        hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().to(device),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().to(device))

        return hidden

We can use the exact same code to train our newly defined LSTM model. Let's try with the same amount of hidden units and 2 LSTM cells.

In [ ]:
                                                                                                                                                                                                                      # Define the model
n_hidden=512
n_layers=2

model = LSTM(chars, device, n_hidden, n_layers).to(device)

# Declaring the hyperparameters
batch_size = 128
seq_length = 100
epochs = 300 # start with 50 or similar if you are debugging
# train much longer if you want good results

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# train the model
train(model, encoded, device, optimizer, criterion, epochs=epochs,
      batch_size=batch_size, seq_length=seq_length)

Epoch: 1/300: Loss: 3.3272:
Epoch: 2/300: Loss: 3.1850:
Epoch: 3/300: Loss: 3.1416:
Epoch: 4/300: Loss: 3.1234:
Epoch: 5/300: Loss: 3.1172:
Epoch: 6/300: Loss: 3.1082:
Epoch: 7/300: Loss: 3.1091:
Epoch: 8/300: Loss: 3.1046:
Epoch: 9/300: Loss: 3.1021:
Epoch: 10/300: Loss: 3.0959:
Epoch: 11/300: Loss: 3.0927:
Epoch: 12/300: Loss: 3.0867:
Epoch: 13/300: Loss: 3.0702:
Epoch: 14/300: Loss: 3.0334:
Epoch: 15/300: Loss: 3.0018:
Epoch: 16/300: Loss: 2.9261:
Epoch: 17/300: Loss: 2.8429:
Epoch: 18/300: Loss: 2.7574:
Epoch: 19/300: Loss: 2.6900:
Epoch: 20/300: Loss: 2.5968:
Epoch: 21/300: Loss: 2.5294:
Epoch: 22/300: Loss: 2.4639:
Epoch: 23/300: Loss: 2.4152:
Epoch: 24/300: Loss: 2.3851:
Epoch: 25/300: Loss: 2.3600:
Epoch: 26/300: Loss: 2.3380:
Epoch: 27/300: Loss: 2.3170:
Epoch: 28/300: Loss: 2.2985:
Epoch: 29/300: Loss: 2.2802:
Epoch: 30/300: Loss: 2.2536:
Epoch: 31/300: Loss: 2.2312:
Epoch: 32/300: Loss: 2.2191:
Epoch: 33/300: Loss: 2.2028:
Epoch: 34/300: Loss: 2.1826:
Epoch: 35/300: Loss: 2.

We can observe that our model is able to achieve a much lower loss than before with our simple RNN implementation. This should now also be reflected when we generate/sample new sonnets.

Again, you can observe that the loss still continues to improve, even after 300 epochs. For the best results, try training the model longer.

In [ ]:
# Generating new text
print(sample(model, 1000, device, prime='A', top_k=5))

Ad my jeyes.

As that a shame which contracked even te..

The praise hours, and yours mort haso as ast,
To bare of that I ab ond a bore wo me.
O! it I not fare fere with my bost,
And so ispainted beauty so such show'st
Thou wilt not, belo your beauty shall exceeme
tfon to hese bastar gofful eresh be detay.
I waren by my bide beasen for thy shen;
Be thoughts, that I have swoet as from thy are.

Bot my dost false in shapen to she hom ;
And course feether is wo deep self you thee.

No e plase whence a will of fairs lefpest,
For dred the barkent, of anquinc prime.
Pess wet a mined wat woth the will,
Wholes is thy gove never being live and all.
What is thy me so proube to alener
Than that yhe dear shall be suck aspearing,
That usur the than when I soughos you stere
This a prafel hours and mores doth thee be.

Then to the bust and welf more light in me
Who that I comperino sham my life decain,
Which his eapy prizide heaven sich make, dhowire.
But deterys raze where beauty love and all,
What 

Arguably there is still discrepancy to the original Shakespeare texts in our just generated examples. However, if we compare this output to our RNN's output, we can see that the LSTM is able to achieve much more consistency in generating proper words as well as sometimes portions of sentences that have improved in terms of grammar. There now seems to be more overall structure.